In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [ ]:
# Cross-shore profiles represented as points; input files to anapa_structure_lines_detection.ipynb
points_all_2013 = gpd.read_file('point_profile_5-6_5m_2013.gpkg')
points_all_2015 = gpd.read_file('point_profile_5-6_5m_2015.gpkg')

In [ ]:
# Structure lines for last date represented as points; output files to anapa_structure_lines_detection.ipynb
crests = gpd.read_file('crests_5-6_2015.gpkg')
toes = gpd.read_file('toes_5-6_2015.gpkg')
toes_landward = gpd.read_file('toes_landward_5-6_2015.gpkg')

In [ ]:
crests['seaward_slope_dZ'] = np.nan
crests['landward_slope_dZ'] = np.nan
crests['seaward_slope_dZ_pos'] = np.nan
crests['seaward_slope_dZ_neg'] = np.nan
crests['landward_slope_dZ_pos'] = np.nan
crests['landward_slope_dZ_neg'] = np.nan
crests['seaward_slope_numpoints'] = np.nan
crests['landward_slope_numpoints'] = np.nan

In [ ]:
for i in crests['LINE_ID']:
    print(i)
    crest_point = crests[crests['LINE_ID'] == i].reset_index()
    toe_point = toes[toes['LINE_ID'] == i].reset_index()
    toe_landward_point = toes_landward[toes_landward['LINE_ID'] == i].reset_index()
    
    if toe_landward_point.empty:
        crests['seaward_slope_dZ'][crests['LINE_ID'] == i] = np.nan
        crests['landward_slope_dZ'][crests['LINE_ID'] == i] = np.nan
    else:
        seaward_slope = points_all_2015[(points_all_2015['ID'] >= toe_point['ID'][0]) & (points_all_2015['ID'] <= crest_point['ID'][0])]
        landward_slope = points_all_2015[(points_all_2015['ID'] > crest_point['ID'][0]) & (points_all_2015['ID'] <= toe_landward_point['ID'][0])]

        seaward_slope_merged = pd.merge(seaward_slope, points_all_2013[points_all_2013['LINE_ID']== i], on=['X', 'Y'], suffixes=('_2015', '_2013'))
        landward_slope_merged = pd.merge(landward_slope, points_all_2013[points_all_2013['LINE_ID']== i], on=['X', 'Y'], suffixes=('_2015', '_2013'))

        seaward_slope_merged['dZ'] = seaward_slope_merged['Z_2015'] - seaward_slope_merged['Z_2013']
        landward_slope_merged['dZ'] = landward_slope_merged['Z_2015'] - landward_slope_merged['Z_2013']

        crests['seaward_slope_dZ'][crests['LINE_ID'] == i] = seaward_slope_merged['dZ'].sum()*0.25
        crests['landward_slope_dZ'][crests['LINE_ID'] == i] = landward_slope_merged['dZ'].sum()*0.25
        
        crests['seaward_slope_dZ_pos'][crests['LINE_ID'] == i] = seaward_slope_merged['dZ'][seaward_slope_merged['dZ'] >= 0].sum()*0.25
        crests['seaward_slope_dZ_neg'][crests['LINE_ID'] == i] = seaward_slope_merged['dZ'][seaward_slope_merged['dZ'] < 0].sum()*0.25
        
        crests['landward_slope_dZ_pos'][crests['LINE_ID'] == i] = landward_slope_merged['dZ'][landward_slope_merged['dZ'] >= 0].sum()*0.25
        crests['landward_slope_dZ_neg'][crests['LINE_ID'] == i] = landward_slope_merged['dZ'][landward_slope_merged['dZ'] < 0].sum()*0.25
        
        crests['seaward_slope_numpoints'][crests['LINE_ID'] == i] = len(seaward_slope_merged)
        crests['landward_slope_numpoints'][crests['LINE_ID'] == i] = len(landward_slope_merged)
 

In [ ]:
crests.to_file('crests_5-6_2015_slopesdZ.gpkg', driver='GPKG')